<a href="https://colab.research.google.com/github/phamthien300820/CS114.K21/blob/master/ML_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
from tqdm import tqdm
import os
import cv2
import pandas as pd
import numpy as np
import h5py
!pip install mahotas
import mahotas
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier


In [6]:
#Đường dẫn output
output_path = "/content/drive/My Drive/X_i1"

# Đường dẫn dữ liệu training
train_path = "/content/drive/My Drive/Machine_Learning Helmet"

# get the training labels
train_labels = os.listdir(train_path)
train_labels.sort()

# fixed-sizes for image
fixed_size = tuple((100, 100))

# bins for histogram
bins = 8

# empty lists to hold feature vectors and labels
global_features = []
labels = []

# feature-descriptor-1: Hu Moments
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

# feature-descriptor-2: Haralick Texture
def fd_haralick(image):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    # return the result
    return haralick

# feature-descriptor-3: Color Histogram
def fd_histogram(image, mask=None):
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

In [7]:
train_labels

['Có người đi xe máy', 'Không có xe máy']

In [8]:
for training_name in train_labels:
    
    dir = os.path.join(train_path, training_name)
    # nhãn đang được train hiện tại
    current_label = training_name
    # Duyệt tất cả các ảnh trong mỗi tập con
    for image in tqdm(os.listdir(dir)):
        
        try:
          path = os.path.join(dir,image)
          #Đọc ảnh và resize lại theo fixed_size
          image = cv2.imread(path)
          image = cv2.resize(image, fixed_size)

          ####################################
          # Global Feature extraction
          ####################################
          fv_hu_moments = fd_hu_moments(image)
          fv_haralick   = fd_haralick(image)
          fv_histogram  = fd_histogram(image)

          ###################################
          # Concatenate global features
          ###################################
          global_feature = np.hstack([fv_histogram, fv_hu_moments, fv_haralick])

          # update the list of labels and feature vectors
          labels.append(current_label)
          global_features.append(global_feature)
        except:
            pass

    print("[STATUS] processed folder: {}".format(current_label))

print("[STATUS] completed Global Feature Extraction...")


# get the overall feature vector size
print ("[STATUS] feature vector size {}".format(np.array(global_features).shape))

# get the overall training label size
print ("[STATUS] training Labels {}".format(np.array(labels).shape))

# encode the target labels
le = LabelEncoder()
target = le.fit_transform(labels)

# normalize the feature vector in the range (0-1)
scaler = MinMaxScaler(feature_range=(0, 1))
rescaled_features = scaler.fit_transform(global_features)

# save the feature vector using HDF5
h5f_data = h5py.File(output_path+'data.h5', 'w')
h5f_data.create_dataset('dataset_1', data=np.array(rescaled_features))

h5f_label = h5py.File(output_path+'labels.h5', 'w')
h5f_label.create_dataset('dataset_1', data=np.array(target))

h5f_data.close()
h5f_label.close()

print("[STATUS] end of training..")

  0%|          | 0/638 [00:00<?, ?it/s]

[STATUS] processed folder: Có người đi xe máy


100%|██████████| 638/638 [02:29<00:00,  4.26it/s]

[STATUS] processed folder: Không có xe máy
[STATUS] completed Global Feature Extraction...
[STATUS] feature vector size (1542, 532)
[STATUS] training Labels (1542,)
[STATUS] end of training..


In [9]:
output_path = "\content\X"

# fixed-sizes for image
#fixed_size = tuple((100, 100))

# Số lượng cây ở RandomForest
num_trees = 300

# bins for histogram
bins = 8

# số hình ảnh mỗi lớp
images_per_class = 10;

# nhập vector đặc trưng và nhãn được training
h5f_data = h5py.File('/content/drive/My Drive/X_i1data.h5', 'r')
h5f_label = h5py.File('/content/drive/My Drive/X_i1labels.h5', 'r')

global_features_string = h5f_data['dataset_1']
global_labels_string = h5f_label['dataset_1']

global_features = np.array(global_features_string)
global_labels = np.array(global_labels_string)

h5f_data.close()
h5f_label.close()


#  Tạo model - Random Forests
clf  = RandomForestClassifier(n_estimators=num_trees)
clf.fit(global_features, global_labels)

# Đường dẫn đến dữ liệu test
test_path = "/content/drive/My Drive/Train_1"
# lấy labels của test
test_labels = os.listdir(test_path)

# sắp xếp labels của test
test_labels.sort()
print(test_labels)
# loop through the test images
test_features = []
test_results = []
for testing_name in test_labels:
    # join the training data path and each species training folder
    dir = os.path.join(test_path, testing_name)
    # get the current training label
    current_label = testing_name
    # loop over the images in each sub-folder
    for image in tqdm(os.listdir(dir)):
        # get the image file name
        try:
          path = os.path.join(dir,image)
          # read the image and resize it oto a fixed-size
          image = cv2.imread(path)
          image = cv2.resize(image, fixed_size)

          ####################################
          # Global Feature extraction
          ####################################
          fv_hu_moments = fd_hu_moments(image)
          fv_haralick   = fd_haralick(image)
          fv_histogram  = fd_histogram(image)

          ###################################
          # Concatenate global features
          ###################################

          # update the list of labels and feature vectors
          test_results.append(current_label)
          test_features.append(np.hstack([fv_histogram, fv_hu_moments, fv_haralick]))
        except:
            pass

# predict label of test image
le = LabelEncoder()
y_result = le.fit_transform(test_results)
y_pred = clf.predict(test_features)

print(y_pred)
print("Result: ", (y_pred == y_result).tolist().count(True)/len(y_result))

0it [00:00, ?it/s]
  0%|          | 0/182 [00:00<?, ?it/s]

['.ipynb_checkpoints', 'Có người đi xe máy', 'Không có xe máy']


100%|██████████| 163/163 [00:25<00:00,  6.38it/s]

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 0 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 0 1 1 1 1 1 0 1 0 0 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 1 1 0 0 0 0
 1 1 1 1 1 0 1 1]
Result:  0.45692883895131087


In [13]:
from sklearn.metrics import classification_report
print(classification_report(y_result, y_pred, labels=[0, 1]))

              precision    recall  f1-score   support

           0       0.44      0.12      0.19       140
           1       0.46      0.83      0.59       127

    accuracy                           0.46       267
   macro avg       0.45      0.47      0.39       267
weighted avg       0.45      0.46      0.38       267

